# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import nltk
import pickle

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report,f1_score,accuracy_score,log_loss

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[nltk_data] Downloading package wordnet to /home/mj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)

Y_labels = ['related', 'request', 'offer', 'aid_related', 
        'medical_help', 'medical_products', 'search_and_rescue', 
        'security', 'military', 'child_alone', 'water', 'food', 
        'shelter', 'clothing', 'money', 'missing_people', 'refugees', 
        'death', 'other_aid', 'infrastructure_related', 'transport', 
        'buildings', 'electricity', 'tools', 'hospitals', 'shops', 
        'aid_centers', 'other_infrastructure', 'weather_related', 
        'floods', 'storm', 'fire', 'earthquake', 'cold', 
        'other_weather', 'direct_report']
X = df['message'].values
Y = df[Y_labels].values
category_names = Y_labels

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    
    #remove punctation
    text = re.sub(r"[^a-zA-Z0-9]",' ',text.lower())
    
    #tokenize text
    tokens = word_tokenize(text)
    
    #lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]    
    
    return (tokens)

#test
tokenize(X[3])

['un',
 'report',
 'leogane',
 '80',
 '90',
 'destroyed',
 'hospital',
 'st',
 'croix',
 'functioning',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(DecisionTreeClassifier(random_state=42),n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test,y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...tion_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
## get f1 score
y_pred = pipeline.predict(X_test)
print("Computing Accuracy for each Category")
for i in range(36):
    print(category_names[i], " Accuracy: ", accuracy_score(y_test[:,i],y_pred[:,i]))
print("\n Classification Report")
print(classification_report(y_test, y_pred, target_names=category_names))

Computing Accuracy for each Category
related  Accuracy:  0.6636779855017169
request  Accuracy:  0.7573445249904617
offer  Accuracy:  0.9919877909194964
aid_related  Accuracy:  0.5331934376192293
medical_help  Accuracy:  0.8668447157573446
medical_products  Accuracy:  0.9135826020602823
search_and_rescue  Accuracy:  0.9471575734452499
security  Accuracy:  0.9692865318580695
military  Accuracy:  0.9395268981304845
child_alone  Accuracy:  1.0
water  Accuracy:  0.8922167111789393
food  Accuracy:  0.8174360930942388
shelter  Accuracy:  0.8401373521556658
clothing  Accuracy:  0.9713849675696299
money  Accuracy:  0.9566959175887066
missing_people  Accuracy:  0.9753910721098817
refugees  Accuracy:  0.9376192293017932
death  Accuracy:  0.9147272033574971
other_aid  Accuracy:  0.7813811522319726
infrastructure_related  Accuracy:  0.8859214040442579
transport  Accuracy:  0.9118657001144601
buildings  Accuracy:  0.9101487981686379
electricity  Accuracy:  0.960129721480351
tools  Accuracy:  0.98836

/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'vect__min_df': [1],
    'vect__lowercase': [False],
    'tfidf__smooth_idf': [False],
}

cv = GridSearchCV(pipeline,param_grid=parameters,cv=2,n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
cv.fit(X_train, y_train)

/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...tion_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__min_df': [1], 'vect__lowercase': [False], 'tfidf__smooth_idf': [False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [9]:
cv.best_score_

0.03143334128309087

In [10]:
## get f1 score
y_pred = cv.predict(X_test)

print("Computing Accuracy for each Category")
for i in range(36):
    print(category_names[i], " Accuracy: ", accuracy_score(y_test[:,i],y_pred[:,i]))
print("\n Classification Report")
print(classification_report(y_test, y_pred, target_names=category_names))

Computing Accuracy for each Category
related  Accuracy:  0.6579549790156429
request  Accuracy:  0.7607783288821061
offer  Accuracy:  0.9914154902708889
aid_related  Accuracy:  0.5335749713849676
medical_help  Accuracy:  0.8605494086226632
medical_products  Accuracy:  0.9152995040061045
search_and_rescue  Accuracy:  0.9481114078595956
security  Accuracy:  0.9671880961465089
military  Accuracy:  0.9414345669591759
child_alone  Accuracy:  1.0
water  Accuracy:  0.8885921404044258
food  Accuracy:  0.8147653567340709
shelter  Accuracy:  0.8416634872186188
clothing  Accuracy:  0.9710034338038916
money  Accuracy:  0.9549790156428843
missing_people  Accuracy:  0.9742464708126669
refugees  Accuracy:  0.9393361312476154
death  Accuracy:  0.9120564669973292
other_aid  Accuracy:  0.790537962609691
infrastructure_related  Accuracy:  0.8826783670354826
transport  Accuracy:  0.9097672644028997
buildings  Accuracy:  0.9095764975200306
electricity  Accuracy:  0.9597481877146128
tools  Accuracy:  0.98740

/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(RandomForestClassifier(random_state = 42), n_jobs = -1)),
#     ('clf', MultiOutputClassifier(KNeighborsClassifier(), n_jobs = -1)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42), n_jobs = -1))
])

parameters = {
    'vect__min_df': [1],
    'vect__lowercase': [False],
    'tfidf__smooth_idf': [False],
}

cv = GridSearchCV(pipeline, parameters, cv = 2, n_jobs = -1)
cv.fit(X_train, y_train)

/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...imator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__min_df': [1], 'vect__lowercase': [False], 'tfidf__smooth_idf': [False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
## get f1 score
y_pred = cv.predict(X_test)
print("Computing Accuracy for each Category")
for i in range(36):
    print(category_names[i], " Accuracy: ", accuracy_score(y_test[:,i],y_pred[:,i]))
print("\n Classification Report")
print(classification_report(y_test, y_pred, target_names=category_names))

Computing Accuracy for each Category
related  Accuracy:  0.756390690576116
request  Accuracy:  0.822014498283098
offer  Accuracy:  0.9948492941625334
aid_related  Accuracy:  0.5930942388401373
medical_help  Accuracy:  0.918733307897749
medical_products  Accuracy:  0.9509729111026326
search_and_rescue  Accuracy:  0.9732926363983212
security  Accuracy:  0.9790156428843952
military  Accuracy:  0.9660434948492942
child_alone  Accuracy:  1.0
water  Accuracy:  0.9391453643647463
food  Accuracy:  0.8897367417016406
shelter  Accuracy:  0.9109118657001145
clothing  Accuracy:  0.9820679130103014
money  Accuracy:  0.9763449065242273
missing_people  Accuracy:  0.9866463181991606
refugees  Accuracy:  0.9608927890118275
death  Accuracy:  0.9523082792827166
other_aid  Accuracy:  0.8695154521175124
infrastructure_related  Accuracy:  0.9334223578786722
transport  Accuracy:  0.9496375429225486
buildings  Accuracy:  0.9481114078595956
electricity  Accuracy:  0.9782525753529188
tools  Accuracy:  0.9944677

/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [13]:
with open('clf.pickle', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.